In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## Import the Datasets and Scale Them

1. Put all dataset names from the datasets folder here
2. change how many timesteps are desired
3. choose the percentage of data that becomes test data

In [ ]:
datasets = [
    '28mar18roof.csv',
    'finaltest_battery34_fieldtest_trail2.csv',
    'finaltest_battery34_fieldtest_trial2.csv',
    'finaltest_battery34_fieldtest_trial3.csv',
    'finaltest_battery34_fieldtest_trial4.csv',
    'finaltest_battery34_fieldtest_trial5.csv',
    'finaltest_battery34_rooftop1.csv',
    'finaltest_battery34_rooftop2.csv',
    'finaltest_battery34_rooftop3.csv',
    'finaltesting_battery34_fieldtest.csv',
    'test_25JAN_rooftop.csv']
datasets = ['datasets/'+ name for name in datasets]
timesteps = 5 * 10   
test_data_percentage = 0.1   # 10% of the data set 

In [ ]:
from data_process import preprocess_and_scale
tr_sets, test_sets, scaler = preprocess_and_scale(
                                    datasets,
                                    test_data_percentage,
                                    timesteps
                                    )

## Building the RNN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

- Build the Structure of the RNN and compile it

In [ ]:
model = Sequential()

# add input layer with some dropout regularisation
model.add(LSTM(units=50, 
              return_sequences=True, 
              input_shape=(timesteps,6)))
model.add(Dropout(0.2))

# add 3 hidden layers
model.add(LSTM(units=50, 
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, 
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, 
               return_sequences=False))
model.add(Dropout(0.2))

# add output layer
model.add(Dense(units=1))

model.compile(optimizer='rmsprop', loss='mean_squared_error')

## Training the Model

In [ ]:
for training_set in tr_sets:
    x_train = training_set[0]
    y_train = training_set[1]
    model.fit(x_train, y_train, epochs = 50)#, batch_size=64)